# Домашнее задание  № 5. Матричные разложения/Тематическое моделирование

### Задание № 1 (4 балла)

Попробуйте матричные разложения с 5 классификаторами - SGDClassifier, KNeighborsClassifier, MultinomialNB, RandomForest, ExtraTreesClassifier (про него подробнее почитайте в документации, он похож на RF). Используйте и NMF и SVD. Сравните результаты на кросс-валидации и выберите лучшее сочетание.

В итоге у вас должно получиться, как минимум 10 моделей (два разложения на каждый классификатор). Используйте 1 и те же параметры кросс-валидации. Параметры векторизации, параметры K в матричных разложениях, параметры классификаторов могут быть разными между экспериментами.

Можете взять поменьше данных, если все будет обучаться слишком долго (не ставьте параметр K слишком большим в NMF, иначе точно будет слишком долго)

In [1]:
!pip install gensim

In [2]:
import gensim
import pandas as pd
import numpy as np
from pymorphy2 import MorphAnalyzer
from collections import Counter
from string import punctuation
from razdel import tokenize as razdel_tokenize
from sklearn.decomposition import TruncatedSVD, NMF, PCA, LatentDirichletAllocation
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.metrics.pairwise import cosine_distances
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from pprint import pprint
import warnings
from matplotlib import pyplot as plt
import seaborn as sns
morph = MorphAnalyzer()
warnings.filterwarnings("ignore")

In [3]:
data = pd.read_csv('data/avito_category_classification.csv')

In [4]:
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    normalized_text = [morph.parse(word)[0].normal_form for word in normalized_text]
    return ' '.join(normalized_text)

In [5]:
data['description_norm'] = data['description'].apply(normalize)

In [6]:
min_df = 5
max_df = 0.1
nmf_size = 200
tokenizer = lambda x: x.split()

In [7]:
pipelines = {
    'SGD_NMF' : Pipeline([
    ('bow', TfidfVectorizer(tokenizer=tokenizer, ngram_range=(1,4), min_df=min_df, max_df=max_df)),
    ('nmf', NMF(nmf_size)),
    ('clf', SGDClassifier(max_iter=1000, tol=1e-3))]),
    'KNeighbors_NMF' : Pipeline([
    ('bow', TfidfVectorizer(tokenizer=tokenizer, ngram_range=(1,4), min_df=min_df, max_df=max_df)),
    ('nmf', NMF(nmf_size)),
    ('clf', KNeighborsClassifier(n_neighbors=5))]),
    'RandomForest_NMF' : Pipeline([
    ('bow', TfidfVectorizer(tokenizer=tokenizer, ngram_range=(1,4), min_df=min_df, max_df=max_df)),
    ('nmf', NMF(nmf_size)),
    ('clf', RandomForestClassifier(max_depth=10, random_state=0))]),
    'ExtraTree_NMF' : Pipeline([
    ('bow', TfidfVectorizer(tokenizer=tokenizer, ngram_range=(1,4), min_df=min_df, max_df=max_df)),
    ('nmf', NMF(nmf_size)),
    ('clf', ExtraTreesClassifier(n_estimators=300, random_state=0))]),
    
    'SGD_SVD' : Pipeline([
    ('bow', TfidfVectorizer(tokenizer=tokenizer, ngram_range=(1,3), min_df=min_df, max_df=max_df)),
    ('svd', TruncatedSVD(500)),
    ('clf', SGDClassifier(max_iter=100, tol=1e-3))]),
    'KNeighbors_SVD' : Pipeline([
    ('bow', TfidfVectorizer(tokenizer=tokenizer, ngram_range=(1,3), min_df=min_df, max_df=max_df)),
    ('svd', TruncatedSVD(500)),
    ('clf', KNeighborsClassifier(n_neighbors=5))]),
    'RandomForest_SVD' : Pipeline([
    ('bow', TfidfVectorizer(tokenizer=tokenizer, ngram_range=(1,3), min_df=min_df, max_df=max_df)),
    ('svd', TruncatedSVD(500)),
    ('clf', RandomForestClassifier(max_depth=10, random_state=0))]),
    'ExtraTree_SVD' : Pipeline([
    ('bow', TfidfVectorizer(tokenizer=tokenizer, ngram_range=(1,3), min_df=min_df, max_df=max_df)),
    ('svd', TruncatedSVD(500)),
    ('clf', ExtraTreesClassifier(n_estimators=300, random_state=0))]),
}

In [8]:
def eval_table(X, y, pipeline, N=6):
    # зафиксируем порядок классов
    labels = list(set(y))
    
    # метрики отдельных фолдов будет хранить в табличке
    fold_metrics = pd.DataFrame(index=labels)
    # дополнительно также соберем таблицу ошибок
    errors = np.zeros((len(labels), len(labels)))
    
    # создаем стратегию кросс-валидации
    # shuffle=True (перемешивание) - часто критично важно указать
    # т.к. данные могут быть упорядочены и модель на этом обучится
    kfold = StratifiedKFold(n_splits=N, shuffle=True, )
    
    for i, (train_index, test_index) in enumerate(kfold.split(X, y)):
        # fit-predict как и раньше, но сразу пайплайном
        pipeline.fit(X[train_index], y[train_index])
        preds = pipeline.predict(X[test_index])
        
        # записываем метрику и индекс фолда
        fold_metrics[f'precision_{i}'] = precision_score(y[test_index], preds, labels=labels, average=None)
        fold_metrics[f'recall_{i}'] = recall_score(y[test_index], preds, labels=labels, average=None)
        fold_metrics[f'f1_{i}'] = f1_score(y[test_index], preds, labels=labels, average=None)
        errors += confusion_matrix(y[test_index], preds, labels=labels, normalize='true')
    
    # таблица для усредненных значений
    # тут мы берем колонки со значениями и усредняем их
    # часто также все метрики сразу суммируют и в конце просто делят на количество фолдов
    # но мы тут помимо среднего также хотим посмотреть на стандартное отклонение
    # чтобы понять как сильно варьируются оценки моделей
    result = pd.DataFrame(index=labels)
    result['precision'] = fold_metrics[[f'precision_{i}' for i in range(N)]].mean(axis=1).round(2)
    result['precision_std'] = fold_metrics[[f'precision_{i}' for i in range(N)]].std(axis=1).round(2)
    
    result['recall'] = fold_metrics[[f'recall_{i}' for i in range(N)]].mean(axis=1).round(2)
    result['recall_std'] = fold_metrics[[f'recall_{i}' for i in range(N)]].std(axis=1).round(2)
    
    result['f1'] = fold_metrics[[f'f1_{i}' for i in range(N)]].mean(axis=1).round(2)
    result['f1_std'] = fold_metrics[[f'f1_{i}' for i in range(N)]].std(axis=1).round(2)
    
    # добавим одну колонку со средним по всем классам
    result.loc['mean'] = result.mean().round(2)
    # проценты ошибок просто усредняем
    errors /= N
    
    return result, errors

In [9]:
metrics_dict = {}

for algo, pipeline in pipelines.items():
    print(algo)
    metrics, errors = eval_table(data['description_norm'], data['category_name'], pipeline)
    metrics_dict[algo] = (metrics, errors)

SGD_NMF
KNeighbors_NMF
RandomForest_NMF
ExtraTree_NMF
SGD_SVD
KNeighbors_SVD
RandomForest_SVD
ExtraTree_SVD


In [10]:
def compare_metrics(difference):
    positives = sum(x > 0 for x in difference)
    negatives = sum(x < 0 for x in difference)
    return int((positives - negatives) >= 0)

In [11]:
def best_metric(metrics):
    metrics_idx = {i: name for i, name in enumerate(metrics.keys())}
    
    curr_metric = 0
    for i in range(1, len(metrics_idx)):
        difference = metrics[metrics_idx[curr_metric]][0].loc['mean'] - metrics[metrics_idx[curr_metric]][0].loc['mean']
        chosen_metric = compare_metrics(difference)
        curr_metric = curr_metric if chosen_metric == 0 else i
    return metrics_idx[curr_metric], metrics[metrics_idx[curr_metric]]

In [12]:
best = best_metric(metrics_dict)

In [13]:
best[0]

'ExtraTree_SVD'

In [14]:
best[1][0]

,precision,precision_std,recall,recall_std,f1,f1_std
Предложение услуг,0.76,0.03,0.69,0.05,0.72,0.03
Мебель и интерьер,0.79,0.07,0.35,0.07,0.48,0.07
Детская одежда и обувь,0.62,0.02,0.75,0.02,0.68,0.01
Квартиры,0.96,0.02,0.95,0.02,0.96,0.01
"Одежда, обувь, аксессуары",0.54,0.02,0.78,0.04,0.64,0.03
Товары для детей и игрушки,0.83,0.06,0.43,0.02,0.57,0.03
Телефоны,0.86,0.05,0.63,0.03,0.73,0.03
Автомобили,0.87,0.03,0.84,0.04,0.86,0.03
Ремонт и строительство,0.68,0.07,0.27,0.04,0.38,0.05
Бытовая техника,0.63,0.12,0.18,0.05,0.28,0.07


В результате обучения всех моделей наилучшим способом по средним метрикам показало себя сочетание разложения SVD и модели ExtraTreesClassifier.

### Задание № 2 (6 баллов)

В Gensim тоже можно добавить нграммы и tfidf. Постройте 1 модель без них (как в семинаре) и еще 3 модели (1 с нграммами, 1 с tfidf и 1 с нграммами и с tfidf). Сранивте качество с помощью метрик (перплексия, когерентность) и на глаз. Определите лучшую модель. Для каждой модели выберите 1 самую красивую на ваш взгляд тему.

Используйте данные википедии из семинара. Можете взять поменьше данных, если все обучается долго.

Важное требование - получившиеся модели не должны быть совсем плохими. Если хороших тем не получается, попробуйте настроить гиперпараметры, отфильтровать словарь по-другому. 

In [15]:
def get_dictionary(texts, no_above=0.1, no_below=10):
    dictinary = gensim.corpora.Dictionary(texts)
    
    dictinary.filter_extremes(no_above=no_above, no_below=no_below)
    dictinary.compactify()
    
    return dictinary

In [16]:
def get_corpus(dictionary, texts):
    return [dictionary.doc2bow(text) for text in texts]

In [17]:
def pipeline_lda(texts, corpus=None, dictinary=None, no_above=0.2, no_below=20, alpha='asymmetric', passes=5):
    if dictinary is None:
        dictinary = get_dictionary(texts, no_above, no_below)
    
    if corpus is None:
        corpus = get_corpus(dictinary, texts)
    
    lda = gensim.models.LdaMulticore(corpus, 
                                 100, # колиество тем
                                 alpha=alpha,
                                 id2word=dictinary, 
                                 passes=passes)
    
    show_metrics(corpus, texts, dictinary, lda)
    
    return lda

In [18]:
def show_metrics(corpus, texts, dictinary, lda):
    print(f'Perplexity: {np.exp2(-lda.log_perplexity(corpus))}')
    
    topics = []
    for topic_id, topic in lda.show_topics(num_topics=100, formatted=False):
        topic = [word for word, _ in topic]
        topics.append(topic)
        
    coherence_model_lda = gensim.models.CoherenceModel(topics=topics, 
                                                   texts=texts, 
                                                   dictionary=dictinary, coherence='c_v')
    
    print(f'Coherence: {coherence_model_lda.get_coherence()}')

In [19]:
def fit_all_models(texts):
    print('Clear model')
    splitted_texts = [text.split() for text in texts]
    lda = pipeline_lda(splitted_texts)
    pprint(lda.print_topics())
    
    print('\nN-gram model')
    texts_ngram = splitted_texts
    ph = gensim.models.Phrases(texts_ngram, scoring='npmi', threshold=0.3) # threshold можно подбирать
    p = gensim.models.phrases.Phraser(ph)
    ngrammed_texts = p[texts_ngram] 
    lda = pipeline_lda(ngrammed_texts)
    pprint(lda.print_topics())
    
    print('\nTfIdf model')
    dictinary = get_dictionary(splitted_texts)
    corpus = get_corpus(dictinary, splitted_texts)
    tfidf = gensim.models.TfidfModel(corpus, id2word=dictinary, )
    corpus = tfidf[corpus]
    lda = pipeline_lda(splitted_texts, corpus, dictinary)
    pprint(lda.print_topics())
    
    print('\nN-gram and Tfidf model')
    dictinary = get_dictionary(ngrammed_texts)
    corpus = get_corpus(dictinary, ngrammed_texts)
    tfidf = gensim.models.TfidfModel(corpus, id2word=dictinary, )
    corpus = tfidf[corpus]
    lda = pipeline_lda(ngrammed_texts, corpus, dictinary)
    pprint(lda.print_topics())

In [20]:
texts = open('data\wiki_data.txt', encoding='utf-8').read().splitlines()[:5000]
texts = ([normalize(text) for text in texts])

In [21]:
fit_all_models(texts)

Clear model
Perplexity: 309.6407181282191
Coherence: 0.5178923636135297
[(99,
  '0.041*"картина" + 0.020*"мария" + 0.019*"портрет" + 0.015*"изобразить" + '
  '0.010*"г" + 0.010*"её" + 0.010*"художник" + 0.009*"у" + 0.008*"пятно" + '
  '0.007*"галерея"'),
 (97,
  '0.012*"или" + 0.009*"система" + 0.008*"мочь" + 0.007*"так" + 0.007*"иметь" '
  '+ 0.006*"использовать" + 0.005*"устройство" + 0.005*"другой" + '
  '0.005*"можно" + 0.004*"такой"'),
 (98,
  '0.039*"марка" + 0.030*"вино" + 0.022*"почтовый" + 0.013*"остров" + '
  '0.013*"город" + 0.010*"выпуск" + 0.006*"под" + 0.005*"почта" + 0.005*"марк" '
  '+ 0.005*"новый"'),
 (96,
  '0.014*"f" + 0.012*"у" + 0.008*"бокс" + 0.007*"век" + 0.007*"l" + '
  '0.007*"залив" + 0.006*"россия" + 0.006*"она" + 0.005*"здание" + '
  '0.005*"город"'),
 (95,
  '0.017*"вид" + 0.012*"яйцо" + 0.010*"личинка" + 0.009*"северный" + '
  '0.009*"звезда" + 0.008*"питаться" + 0.007*"тело" + 0.007*"площадь" + '
  '0.007*"самка" + 0.006*"встречаться"'),
 (94,
  '0.011*"

Perplexity: 171254.5062983833
Coherence: 0.38636217702715103
[(99,
  '0.002*"полигон" + 0.001*"северный_ирландия" + 0.001*"нил" + '
  '0.001*"сельсовет" + 0.001*"танк" + 0.001*"кормовой" + 0.000*"брак" + '
  '0.000*"личный_состав" + 0.000*"эскадренный_миноносец" + 0.000*"патриарх"'),
 (98,
  '0.000*"матч" + 0.000*"сборная" + 0.000*"нация" + 0.000*"уэльс" + '
  '0.000*"ирландия" + 0.000*"с_счёт" + 0.000*"выиграть" + 0.000*"турнир" + '
  '0.000*"крым" + 0.000*"двор"'),
 (97,
  '0.000*"северный_кавказ" + 0.000*"кавказский" + 0.000*"республика" + '
  '0.000*"э" + 0.000*"до_н" + 0.000*"памятник" + 0.000*"грузинский" + '
  '0.000*"чечня" + 0.000*"чеченский" + 0.000*"эпоха"'),
 (95,
  '0.001*"быть_убить" + 0.000*"убийца" + 0.000*"бандит" + 0.000*"сергей" + '
  '0.000*"группировка" + 0.000*"расстрелять" + 0.000*"владимир" + '
  '0.000*"убийство" + 0.000*"завод" + 0.000*"предприятие"'),
 (96,
  '0.015*"теорема" + 0.008*"квантовый" + 0.002*"пароход" + 0.001*"теория" + '
  '0.001*"полузащитник" +

По оценке метриками и "на глаз" самые хорошие результаты получаются у чистой модели. Чуть хуже проявляет себя модель с н-граммами. У моделей, где есть TfIdf получаются наихудшие результаты, но тем не менее, среди этих результатов тоже можно выделить какую-то обшую тему. Вероятно, такие результаты связаны с тем, что корпус для обучения взят не такой большой. Также уменьшение количества тем улучшает качество моделирование по метрикам.